In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import os
os.chdir('..')
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings

In [2]:
from langchain_chinese import (
    create_qa_chain,
    create_qa_toolkits,
    create_reason_agent,
)
from langchain_chinese.langgraph import create_tool_calling_executor

from langchain.schema import Document
query_document = Document(page_content="langchain_chinese 是为中国大语言模型优化的langchain模块")

from langchain_community.vectorstores import Chroma
db = Chroma.from_documents([query_document], ZhipuAIEmbeddings())

llm_zhipu = ChatZhipuAI(model="glm-4")

chain = create_qa_chain(llm_zhipu, db.as_retriever())
tools = create_qa_toolkits(qa_chain=chain)

In [3]:
from langchain.tools import tool
import random

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下", "在书架中", "在阳台"])

In [4]:
# from langchain_openai import ChatOpenAI
# model = ChatZhipuAI()
app = create_tool_calling_executor(llm_zhipu, [*tools, where_is_cat_hiding])

In [8]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="请查询资料，告诉我langchain_chinese是什么？")]}
async for s in app.astream(inputs):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8465305361399317318', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'ask_document'}, 'type': 'function'}]})]}}
----


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'action': {'messages': [ToolMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是为了更好地与中国的语言环境和需求相适应，提高大语言模型在中文语境下的表现和应用效果。', tool_call_id='call_8465305361399317318')]}}
----
{'agent': {'messages': [AIMessage(content='根据我的了解，langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是为了更好地与中国的语言环境和需求相适应，提高大语言模型在中文语境下的表现和应用效果。')]}}
----
{'__end__': {'messages': [HumanMessage(content='请查询资料，告诉我langchain_chinese是什么？'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8465305361399317318', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'ask_document'}, 'type': 'function'}]}), ToolMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是为了更好地与中国的语言环境和需求相适应，提高大语言模型在中文语境下的表现和应用效果。', tool_call_id='call_8465305361399317318'), AIMessage(content='根据我的了解，langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是为了更好地与中国的语言环境和需求相适应，提高大语言模型在中文语境下的表现和应用效果。')]}}
----


In [116]:
inputs = {"messages": [HumanMessage(content="猫在哪里？")]}
async for s in app.astream_events(inputs, version="v1", include_names=["call_model"]):
    event = s['event']
    name = s['name']
    tags = s['tags']
    metadata = s['metadata']
    data = s['data']
    if(event == "on_chain_stream"):
        print(event, name, tags)
        print(data)
    else:
        print(event)


on_chain_start
on_chain_stream call_model ['seq:step:2']
{'chunk': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8465308385056330157', 'function': {'arguments': '{"idea":"猫在哪里"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]})]}}
on_chain_end
on_chain_start
on_chain_stream call_model ['seq:step:2']
{'chunk': {'messages': [AIMessage(content='猫在阳台上。')]}}
on_chain_end


In [118]:
inputs =  {"messages": [HumanMessage(content="霍金的生日是哪一天？")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")


Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8465304193168110376', 'function': {'arguments': '{"query":"霍金的生日是哪一天？"}', 'name': 'ask_document'}, 'type': 'function'}]})]}

---



Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Output from node 'action':
---
{'messages': [ToolMessage(content='霍金的生日是1942年1月8日。', tool_call_id='call_8465304193168110376')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='根据资料库查询，霍金的生日是1942年1月8日。')]}

---

Output from node '__end__':
---
{'messages': [HumanMessage(content='霍金的生日是哪一天？'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8465304193168110376', 'function': {'arguments': '{"query":"霍金的生日是哪一天？"}', 'name': 'ask_document'}, 'type': 'function'}]}), ToolMessage(content='霍金的生日是1942年1月8日。', tool_call_id='call_8465304193168110376'), AIMessage(content='根据资料库查询，霍金的生日是1942年1月8日。')]}

---

